In [1]:
import numpy as np
import cv2
import tkinter as tk
from PIL import Image, ImageTk
import random


In [2]:
import random

# Función para generar un color aleatorio en el rango RGB
def generate_random_color():
    return [random.randint(0, 255) for _ in range(3)]

In [3]:
# Función para crear una imagen del gato con un color dado
def crear_imagen_gato(color):
    # Crear una imagen del gato con el color dado
#     height = random.randint(0,100)
#     width = random.randint(0,100)
    height, width = 100, 120
#     height, width = tuple(size)
    image = np.ones((height, width, 3), dtype=np.uint8) * 255  # Fondo blanco

    # Dibujar la cabeza del gato
    cv2.ellipse(image, (70, 50), (30, 20), 0, 0, 360, tuple(color), -1)

    # Dibujar los ojos
    cv2.circle(image, (55, 45), 5, (0, 0, 0), -1)
    cv2.circle(image, (85, 45), 5, (0, 0, 0), -1)

    # Dibujar la nariz
    cv2.ellipse(image, (70, 55), (5, 3), 0, 0, 360, (0, 0, 0), -1)

    # Dibujar las orejas
    ear_left = np.array([[50, 30], [60, 20], [70, 30]])
    ear_right = np.array([[90, 30], [80, 20], [70, 30]])
    cv2.fillPoly(image, [ear_left], tuple(color))
    cv2.fillPoly(image, [ear_right], tuple(color))

    return image

In [4]:
import math

# Función para calcular la distancia euclidiana entre dos colores en el espacio RGB
def euclidean_distance(color1, color2):
    return math.sqrt(sum((c1 - c2) ** 2 for c1, c2 in zip(color1, color2)))

In [5]:
# Función para generar una población inicial de colores aleatorios
def generate_initial_population(population_size):
    return [generate_random_color() for _ in range(population_size)]

In [6]:
# Función de selección de torneo
def tournament_selection(population, target_color,k=3):
    tournament_contestants = random.sample(population, k)
    return min(tournament_contestants, key=lambda x: euclidean_distance(x,target_color))

In [7]:
# Función de cruce (crossover) de un punto
def single_point_crossover(parent1, parent2):
    crossover_point = random.randint(0, len(parent1) - 1)
    child = parent1[:crossover_point] + parent2[crossover_point:]
    return child

# Ejemplo de cruce
# parent1 = [100, 150, 200]
# parent2 = [50, 200, 100]
# child = single_point_crossover(parent1, parent2)
# print("Descendiente generado mediante cruce:", child)

In [8]:
# Función de mutación
def mutation(individual, mutation_rate=0.8):
    mutated_individual = individual[:]
    if random.random() < mutation_rate:
        mutation_index = random.randint(0, len(individual) - 1)
        mutated_individual[mutation_index] = random.randint(0, 255)
    return mutated_individual

# Ejemplo de mutación
# individual = [200, 100, 51]
# mutated_individual = mutation(individual)
# print("Individuo mutado:", mutated_individual)

In [9]:
# Función para evolucionar la población durante una generación de manera generacional
def evolve_population(population,target_color, mutation_rate=0.8, crossover_rate=0.6):
    new_population = []

    # Cantidad de individuos que se seleccionarán para el cruce y la mutación
    crossover_count = int(len(population) * crossover_rate)
    mutation_count = len(population) - crossover_count

    # Realizar cruce
    for _ in range(crossover_count):
        parent1 = tournament_selection(population,target_color)
        parent2 = tournament_selection(population,target_color)
        child = single_point_crossover(parent1, parent2)
        new_population.append(child)

    # Realizar mutación
    for _ in range(mutation_count):
        individual = tournament_selection(population,target_color)
        mutated_individual = mutation(individual, mutation_rate)
        new_population.append(mutated_individual)

    return new_population

In [10]:
# Función para determinar si se ha alcanzado el criterio de parada
def has_converged(population, target_color, threshold=1):
    return all(euclidean_distance(individual, target_color) < threshold for individual in population)

In [11]:
# Función para actualizar el color de un gato en la interfaz gráfica
def update_cat_color(canvas, cat_index, new_color, width_increment=10, height_increment=10):
    global imagenes
    global cat_photos

    # Crear una nueva imagen del gato con el nuevo color
    size = (random.randint(0,255),random.randint(0,255))
    image = crear_imagen_gato(new_color)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)
    size = (image.width + width_increment, image.height + height_increment)
#     print(size) 
    image = image.resize(size)
    photo = ImageTk.PhotoImage(image)

    # Actualizar la lista de fotos y la imagen del gato en el lienzo
    imagenes[cat_index] = photo
    canvas.itemconfig(cat_photos[cat_index], image=photo)

In [12]:
# Función para evolucionar el color de los gatos en la interfaz gráfica en tiempo real
import time
def evolve_cat_colors(canvas, target_color,poblacion, population_size=10, mutation_rate=0.9, max_generations=1000):
    global imagenes

    # Inicializar la población de colores para los gatos
    population = poblacion

    # Evolucionar la población y actualizar los colores de los gatos en la interfaz gráfica en tiempo real
    generations = 0
    best_color = None
    while not has_converged(population, target_color) and generations < max_generations:
        # Seleccionar el mejor color de la población actual
        population = evolve_population(population,target_color)
        best_color = min(population, key=lambda x: euclidean_distance(x, target_color))

        # Actualizar los colores de los gatos en la interfaz gráfica en tiempo real
        for i, color in enumerate(population):
            update_cat_color(canvas, i, color)
            canvas.update()  # Actualizar la interfaz gráfica en cada generación
            
#             time.sleep(0.005) 
        time.sleep(1) 


In [13]:
# Parámetros
target_color = [255, 0, 0]
population_size = 10
mutation_rate = 0.8
max_generations = 10000
# population = generate_initial_population(population_size)

In [18]:
# Crear la ventana de la interfaz
root = tk.Tk()
root.title("Gatos!")

# Crear un lienzo para colocar las imágenes de los gatos
canvas = tk.Canvas(root, width=1000, height=1200)
canvas.pack()

# Lista para almacenar las imágenes y evitar que Python las elimine
imagenes = []

# Lista para almacenar los objetos PhotoImage de los gatos
cat_photos = []

# Coordenadas iniciales para la primera imagen de gato
x, y = 110, 50
population = generate_initial_population(population_size)
# Crear 10 imágenes de gatos y colocarlas en el lienzo
for i,color in enumerate(population):
    # Crear una imagen de gato inicial con un color aleatorio
#     color = generate_random_color()
    size = (random.randint(0,255),random.randint(0,255))
    image = crear_imagen_gato(color)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)
    photo = ImageTk.PhotoImage(image)

    # Agregar la imagen a la lista para mantener la referencia
    imagenes.append(photo)

    # Colocar la imagen en el lienzo y almacenar el objeto PhotoImage
    cat_photo = canvas.create_image(x, y, image=photo)
    cat_photos.append(cat_photo)

    # Ajustar las coordenadas para la próxima imagen
    x += 100  # Aumentar la coordenada y para separar las imágenes
    if x >= 800:
        x = 300
        y += 150

# Color objetivo al que queremos que evolucionen los gatos
target_color = [0, 0, 255]

# Evolucionar los colores de los gatos hacia el color objetivo en tiempo real
evolve_cat_colors(canvas, target_color,population)
# Mostrar ventana
root.mainloop()

ValueError: too many values to unpack (expected 2)

In [16]:
import random
import math
import numpy as np
import cv2
from PIL import Image, ImageTk
import tkinter as tk

# Función para generar un tamaño de imagen aleatorio
def generate_random_size():
    return (random.randint(50, 200), random.randint(50, 200))  # Se puede ajustar el rango según sea necesario

# Función para generar una población inicial de tamaños de imagen aleatorios
def generate_initial_population_size(population_size):
    return [generate_random_size() for _ in range(population_size)]

# Función de mutación para ajustar aleatoriamente el tamaño de la imagen
def mutation(individual, mutation_rate=0.8):
    mutated_individual = list(individual)  # Convertir la tupla a lista para permitir la mutación
    if random.random() < mutation_rate:
        # Ajustar aleatoriamente el ancho y el alto de la imagen
        mutated_individual[0] = max(10, individual[0] + random.randint(-20, 20))
        mutated_individual[1] = max(10, individual[1] + random.randint(-20, 20))
    return tuple(mutated_individual)  # Convertir la lista de vuelta a tupla

# Función para determinar si se ha alcanzado el criterio de parada (tamaño de imagen cercano al objetivo)
def has_converged(population, target_size, threshold=10):
    return all(abs(individual[0] - target_size[0]) < threshold and abs(individual[1] - target_size[1]) < threshold for individual in population)

# Función para evolucionar la población durante una generación de manera generacional
def evolve_population(population, target_size, mutation_rate=0.8, crossover_rate=0.6):
    new_population = []

    # Cantidad de individuos que se seleccionarán para el cruce y la mutación
    crossover_count = int(len(population) * crossover_rate)
    mutation_count = len(population) - crossover_count

    # Realizar cruce
    for _ in range(crossover_count):
        parent1 = tournament_selection(population, target_size)
        parent2 = tournament_selection(population, target_size)
        child = single_point_crossover(parent1, parent2)
        new_population.append(child)

    # Realizar mutación
    for _ in range(mutation_count):
        individual = tournament_selection(population, target_size)
        mutated_individual = mutation(individual, mutation_rate)
        new_population.append(mutated_individual)

    return new_population


# Función para evolucionar el tamaño de las imágenes de los gatos en la interfaz gráfica en tiempo real
def evolve_cat_sizes(canvas, target_size, population_size=10, mutation_rate=0.9, max_generations=1000):
    # Inicializar la población de tamaños para los gatos
    population = generate_initial_population(population_size)

    # Evolucionar la población y actualizar los tamaños de las imágenes de los gatos en la interfaz gráfica en tiempo real
    generations = 0
    while not has_converged(population, target_size) and generations < max_generations:
        population = evolve_population(population, target_size)
        # Actualizar los tamaños de las imágenes de los gatos en la interfaz gráfica en tiempo real
        for i, size in enumerate(population):
            update_cat_size(canvas, i, size)
            canvas.update()  # Actualizar la interfaz gráfica en cada generación
            time.sleep(0.1)  # Agregar un pequeño retraso entre cada iteración para visualizar el cambio
        generations += 1


# Función para actualizar el tamaño de la imagen del gato en la interfaz gráfica
def update_cat_size(canvas, cat_index, new_size):
    global imagenes
    global cat_photos

    # Crear una nueva imagen del gato con el nuevo tamaño
    image = crear_imagen_gato(new_size)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = Image.fromarray(image)
    photo = ImageTk.PhotoImage(image)

    # Actualizar la lista de fotos y la imagen del gato en el lienzo
    imagenes[cat_index] = photo
    canvas.itemconfig(cat_photos[cat_index], image=photo)

# Función para crear una imagen del gato con un tamaño dado
def crear_imagen_gato(size):
    # Crear una imagen del gato con el tamaño dado
    height, width = size
    image = np.ones((height, width, 3), dtype=np.uint8) * 255  # Fondo blanco

    # Dibujar la cabeza del gato
    cv2.ellipse(image, (int(width / 2), int(height / 2)), (int(width / 4), int(height / 3)), 0, 0, 360, (0, 0, 0), -1)

    # Dibujar los ojos
    cv2.circle(image, (int(width * 0.35), int(height * 0.4)), int(width * 0.05), (255, 255, 255), -1)
    cv2.circle(image, (int(width * 0.65), int(height * 0.4)), int(width * 0.05), (255, 255, 255), -1)

    # Dibujar la nariz
    cv2.ellipse(image, (int(width / 2), int(height * 0.55)), (int(width * 0.1), int(height * 0.1)), 0, 0, 360, (255, 0, 0), -1)

    # Dibujar las orejas
    ear_left = np.array([[int(width * 0.2), int(height * 0.15)], [int(width * 0.3), int(height * 0.05)], [int(width * 0.4), int(height * 0.15)]])
    ear_right = np.array([[int(width * 0.6), int(height * 0.15)], [int(width * 0.7), int(height * 0.05)], [int(width * 0.8), int(height * 0.15)]])
    cv2.fillPoly(image, [ear_left], (0, 0, 0))
    cv2.fillPoly(image, [ear_right], (0, 0, 0))

    return image

# Función para seleccionar un individuo de la población utilizando selección de torneo
def tournament_selection(population, target_size, k=3):
    tournament_contestants = random.sample(population, k)
    return min(tournament_contestants, key=lambda x: abs(x[0] - target_size[0]) + abs(x[1] - target_size[1]))

# Función de cruce (crossover) de un punto
def single_point_crossover(parent1, parent2):
    crossover_point = random.randint(0, 1)  # Seleccionar un punto de cruce (0 para el ancho, 1 para el alto)
    child = list(parent1)
    child[crossover_point] = parent2[crossover_point]
    return tuple(child)

# Función principal
def main():
    # Parámetros
    target_size = (150, 200)
    population_size = 10
    mutation_rate = 0.8
    max_generations = 10000

    # Crear la ventana de la interfaz
    root = tk.Tk()
    root.title("Gatos!")

    # Crear un lienzo para colocar las imágenes de los gatos
    canvas = tk.Canvas(root, width=1000, height=1200)
    canvas.pack()

    # Lista para almacenar las imágenes y evitar que Python las elimine
    imagenes = []

    # Lista para almacenar los objetos PhotoImage de los gatos
    cat_photos = []

    # Coordenadas iniciales para la primera imagen de gato
    x, y = 110, 50

    # Crear 10 imágenes de gatos y colocarlas en el lienzo
    for _ in range(population_size):
        # Crear una imagen de gato inicial con un tamaño aleatorio
        size = generate_random_size()
        image = crear_imagen_gato(size)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)
        photo = ImageTk.PhotoImage(image)

        # Agregar la imagen a la lista para mantener la referencia
        imagenes.append(photo)

        # Colocar la imagen en el lienzo y almacenar el objeto PhotoImage
        cat_photo = canvas.create_image(x, y, image=photo)
        cat_photos.append(cat_photo)

        # Ajustar las coordenadas para la próxima imagen
        x += 100  # Aumentar la coordenada y para separar las imágenes
        if x >= 800:
            x = 300
            y += 150

    # Evolucionar los tamaños de las imágenes de los gatos hacia el tamaño objetivo en tiempo real
    evolve_cat_sizes(canvas, target_size, population_size, mutation_rate, max_generations)

    # Mostrar ventana
    root.mainloop()

if __name__ == "__main__":
    main()


TclError: image "pyimage132" doesn't exist

In [17]:
size = generate_initial_population(10)
color = generate_initial_population_size(10)
print(size)
print(color)
for i,(size,color) in enumerate(zip(size,color)):
    print(size)
    print(color)

[[213, 107, 4], [80, 156, 122], [229, 149, 145], [47, 210, 45], [206, 64, 149], [221, 208, 52], [218, 112, 14], [81, 209, 121], [52, 17, 235], [25, 211, 5]]
[(124, 146), (175, 122), (155, 161), (180, 61), (115, 131), (54, 88), (55, 149), (132, 178), (100, 65), (74, 187)]
[213, 107, 4]
(124, 146)
[80, 156, 122]
(175, 122)
[229, 149, 145]
(155, 161)
[47, 210, 45]
(180, 61)
[206, 64, 149]
(115, 131)
[221, 208, 52]
(54, 88)
[218, 112, 14]
(55, 149)
[81, 209, 121]
(132, 178)
[52, 17, 235]
(100, 65)
[25, 211, 5]
(74, 187)
